### Installing the Package

In [1]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


### Connect to a database

In [2]:
from sqlalchemy import create_engine

connection_string = 'sqlite:///winkel.db' 
engine = create_engine(connection_string, echo=False)


### Create a Table

In [3]:
from sqlalchemy import Table, Column, Integer, Float, String, MetaData
meta = MetaData()

producten = Table(
   'producten', meta, 
   Column('id', Integer, primary_key = True), 
   Column('product_naam', String), 
   Column('kostprijs', Float), 
    Column('verkoopprijs', Float), 
)

meta.drop_all(engine)
meta.create_all(engine)


### Filling a table with data(Insert)

In [4]:
insert = producten.insert().values(product_naam = 'Wasmand', kostprijs = '1.5', verkoopprijs = '4.5')
with engine.connect() as connection:
    connection.execute(insert)

In [5]:
insert = producten.insert().values({ 'product_naam': 'Wasmand', 'kostprijs': 2, 'verkoopprijs': 5.5 })
with engine.connect() as connection:
    connection.execute(insert)

In [6]:
insert = producten.insert().values(
    [
        { 'product_naam': 'Wasmand', 'kostprijs': 2, 'verkoopprijs': 5.5 },
        { 'product_naam': 'Droogrek', 'kostprijs': 10, 'verkoopprijs': 15 }
    ])
with engine.connect() as connection:
    connection.execute(insert)

### Select data from one or more tables(Select)

In [7]:
s = producten.select()
with engine.connect() as connection:
    result = connection.execute(s)
    for row in result:
        print(row)

(1, 'Wasmand', 1.5, 4.5)
(2, 'Wasmand', 2.0, 5.5)
(3, 'Wasmand', 2.0, 5.5)
(4, 'Droogrek', 10.0, 15.0)


In [8]:
from sqlalchemy import select

s = select([producten]).where(producten.c.product_naam == 'Wasmand')
with engine.connect() as connection:
    result = connection.execute(s)
    for row in result:
        print(row)

(1, 'Wasmand', 1.5, 4.5)
(2, 'Wasmand', 2.0, 5.5)
(3, 'Wasmand', 2.0, 5.5)


In [9]:
import pandas as pd

with engine.connect() as connection:
    df = pd.read_sql('producten', connection)

In [10]:
import pandas as pd

with engine.connect() as connection:
    df = pd.read_sql('producten', connection)
df = df.groupby('product_naam').mean()
df

,id,kostprijs,verkoopprijs
product_naam,,,
Droogrek,4,10.000000,15.000000
Wasmand,2,1.833333,5.166667


### Change and delete existing data(Update & Delete)

In [11]:
update = producten.update().values({ 'kostprijs': 3, 'verkoopprijs': 6 }).where(producten.c.product_naam == 'Wasmand')
with engine.connect() as connection:
    connection.execute(update)
    df = pd.read_sql('producten', connection)            
df

,id,product_naam,kostprijs,verkoopprijs
0,1,Wasmand,3.0,6.0
1,2,Wasmand,3.0,6.0
2,3,Wasmand,3.0,6.0
3,4,Droogrek,10.0,15.0


In [12]:
delete = producten.delete().where(producten.c.verkoopprijs < 10)
with engine.connect() as connection:
    connection.execute(delete)
    df = pd.read_sql('producten', connection)  